## 数据预处理

In [1]:
import os
import cv2
import h5py
import numpy as np
import pandas as pd
from keras.applications import Xception, xception
from keras.models import Model
from keras.layers import Dropout, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
data_path_train = '../dogs-vs-cats-dataset/train'
data_path_train_extra = '../dogs-vs-cats-dataset/images-Oxford-IIIT'
data_path_test = '../dogs-vs-cats-dataset/test'
image_names_train = os.listdir(data_path_train)
image_names_train_extra = os.listdir(data_path_train_extra)
image_names_test = os.listdir(data_path_test)
input_shape = (299, 299, 3)
labels = []
trains = []
tests = []


# 处理标准的训练数据
for i in tqdm(range(len(image_names_train))):
    image_name = image_names_train[i]
    image_path = os.path.join(data_path_train, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    trains.append(image[:, :, ::-1])
    # cat: 0, dog: 1
    category = 1 if 'dog' in image_name else 0
    labels.append(category)

    
# 猫的种类
cat_types = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian',
             'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']
# 处理扩展的训练数据
for i in tqdm(range(len(image_names_train_extra))):
    image_name = image_names_train_extra[i]
    image_path = os.path.join(data_path_train_extra, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read extra train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    index = len(image_names_train) + i
    trains.append(image[:, :, ::-1])
    
    # 获取动物的种类（dog or cat）
    spt = image_names_train_extra[i].split('_')
    spt.pop()
    tp = '_'.join(spt)
    category = 0 if tp in cat_types else 1
    labels.append(category)
    

# 处理标准的测试数据
for i in tqdm(range(len(image_names_test))):
    image_name = image_names_test[i]
    image_path = os.path.join(data_path_test, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read test image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    tests.append(image[:, :, ::-1])
    
    
trains = np.array(trains)
labels = np.array(labels)
tests = np.array(tests)

print('Training data size: %d' % len(trains))
print('Label size: %d' % len(labels))
print('Testing data size: %d' % len(tests))

 12%|█▏        | 881/7393 [00:04<00:33, 193.34it/s]

Read extra train image failed: ../dogs-vs-cats-dataset/images-Oxford-IIIT/Abyssinian_102.mat


 14%|█▍        | 1027/7393 [00:05<00:32, 193.94it/s]

Read extra train image failed: ../dogs-vs-cats-dataset/images-Oxford-IIIT/Abyssinian_100.mat


 19%|█▉        | 1426/7393 [00:07<00:30, 195.65it/s]

...

Read extra train image failed: ../dogs-vs-cats-dataset/images-Oxford-IIIT/Abyssinian_34.jpg


 98%|█████████▊| 7281/7393 [00:37<00:00, 194.57it/s]

Read extra train image failed: ../dogs-vs-cats-dataset/images-Oxford-IIIT/Egyptian_Mau_167.jpg


100%|██████████| 12500/12500 [00:36<00:00, 340.60it/s]


Training data size: 32384
Label size: 32384
Testing data size: 12500


## 特征提取

In [ ]:
x = Input(shape=input_shape)
x = Lambda(xception.preprocess_input)(x)
model = Xception(input_tensor=x, input_shape=input_shape, weights='imagenet', include_top=False, pooling='avg')
bottleneck_features_train = model.predict(trains, batch_size=128)
bottleneck_features_test = model.predict(tests, batch_size=128)

with h5py.File("bottleneck_features.h5", 'w') as h:
    h.create_dataset('trains', data=bottleneck_features_train)
    h.create_dataset('labels', data=labels)
    h.create_dataset('tests', data=bottleneck_features_test)

print('bottleneck features have been wrote to bottleneck_features.h5')

## 构建模型

In [9]:
with h5py.File('bottleneck_features.h5','r') as h:
    X_train = np.array(h['trains'])
    y_train = np.array(h['labels'])
    X_test = np.array(h['tests'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True, test_size=0.2, random_state=2018)

x = Input(shape=(X_train.shape[1],))
y = Dropout(0.3)(x)
y = Dense(1, activation='sigmoid')(y)
model = Model(x, y)

model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

print('Model ready!')

Model ready!


## 训练

In [10]:
model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

(37,)
(29, 2048)
(29,)
(8, 2048)
(8,)
Train on 29 samples, validate on 8 samples
Epoch 1/10
29/29 [==============================] - 1s 26ms/step - loss: 0.6829 - acc: 0.5172 - val_loss: 0.6570 - val_acc: 0.7500
Epoch 2/10
29/29 [==============================] - 0s 954us/step - loss: 0.5965 - acc: 0.6897 - val_loss: 0.6048 - val_acc: 0.7500
Epoch 3/10
29/29 [==============================] - 0s 426us/step - loss: 0.5337 - acc: 0.9655 - val_loss: 0.5539 - val_acc: 0.8750
Epoch 4/10
29/29 [==============================] - 0s 809us/step - loss: 0.4718 - acc: 0.9310 - val_loss: 0.5105 - val_acc: 0.8750
Epoch 5/10
29/29 [==============================] - 0s 627us/step - loss: 0.4335 - acc: 0.9655 - val_loss: 0.4763 - val_acc: 0.8750
Epoch 6/10
29/29 [==============================] - 0s 621us/step - loss: 0.3832 - acc: 1.0000 - val_loss: 0.4489 - val_acc: 0.8750
Epoch 7/10
29/29 [==============================] - 0s 322us/step - loss: 0.3377 - acc: 1.0000 - val_loss: 0.4252 - val_acc: 0.8

## 测试

In [12]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

df = pd.read_csv("sample_submission.csv")

for i in range(len(image_names_test)):
    image_name = image_names_test[i]
    index = int(str.split(image_name, '.')[0]) - 1
    df.iat[index, 1] = y_pred[i]

df.to_csv('predict.csv', index=None)
print('The prediction result has been wrote to predict.csv')

15/15 [==============================] - 0s 129us/step
The prediction result has been wrote to predict.csv
